# Script/notebook to assess the quality of a marconi copy directory

In [5]:
import pandas as pd
import numpy as np
import pathlib as pth
import sys
import os
from datetime import datetime
sys.path.append('~/coding/projects/flopter')
import flopter.spice.inputparser as inp

In [29]:
# path_to_sims = pth.Path('/home/jleland/data/external/spice/marconi')
# path_to_sims = pth.Path('/home/jleland/data/external/spice/marconi')
# groups = list(path_to_sims.glob('*'))

paths_to_sims = [
    pth.Path('/home/jleland/data/external/spice/marconi'),
    pth.Path('/home/jleland/data/external_big/spice/marconi'),
    pth.Path('/home/jleland/data/external_big/spice/marconi_scratch'),
]
groups = []
[groups.extend(list(pts.glob('*'))) for pts in paths_to_sims]

print(groups)

[PosixPath('/home/jleland/data/external/spice/marconi/lowdens_anglescan'), PosixPath('/home/jleland/data/external/spice/marconi/restart_test'), PosixPath('/home/jleland/data/external_big/spice/marconi/highdens_anglescan'), PosixPath('/home/jleland/data/external_big/spice/marconi/lowdens_anglescan'), PosixPath('/home/jleland/data/external_big/spice/marconi/magnum'), PosixPath('/home/jleland/data/external_big/spice/marconi/middens_anglescan'), PosixPath('/home/jleland/data/external_big/spice/marconi/middens_anglescan_failed'), PosixPath('/home/jleland/data/external_big/spice/marconi/restart_test'), PosixPath('/home/jleland/data/external_big/spice/marconi/test'), PosixPath('/home/jleland/data/external_big/spice/marconi_scratch/highdens_anglescan'), PosixPath('/home/jleland/data/external_big/spice/marconi_scratch/lowdens_anglescan'), PosixPath('/home/jleland/data/external_big/spice/marconi_scratch/middens_anglescan'), PosixPath('/home/jleland/data/external_big/spice/marconi_scratch/rprobe_

In [12]:
str(datetime.fromtimestamp(groups[0].stat().st_ctime))

'2020-01-09 12:44:31'

In [35]:
def print_run_dir_stats(run, depth=1):
    tab = "\t"
    print(r)
    print(f'{"".join([tab for i in range(depth)])}-> mat_files: {len(list(run.glob("*.mat")))}, {len(list(run.glob("backup_*")))}')

def print_backups(run):
    backups = list(run.glob('backup_*'))
    for backup in backups:
        print(f'\t-> {backup.name}')
        print_run_dir_stats(backup, depth=2)

In [16]:
skippable_groups = {'restart_test'}
skippable_runs = {'inpartest_as'}

In [17]:
for group in groups:
    if group.name in skippable_groups:
        continue
    print(f'\n{group.name}')
    for i, run in enumerate(group.glob('*')):
        if run.name in skippable_runs:
            continue
        print(i, run.name)
        backups = list(run.glob('backup_*'))
        scan_runs = list(run.glob('alpha_yz*'))
        print_run_dir_stats(run)
        if len(backups) > 0:
            # if there are backups it's probbaly a run
            print_backups(run)
        elif len(scan_runs) > 0:
            for scan_run in scan_runs:
                print(f' - {scan_run.name}')
                print_run_dir_stats(scan_run)
                print_backups(scan_run)
    


lowdens_anglescan
1 angled_tiltscan


NameError: name 'print_run_dir_stats' is not defined

In [18]:
dir_df = pd.DataFrame(columns=['directory', 'group', 'scan', 'run', 'mat_files', 'backups', 'empty_bups', 'last_working_bup', 'date_copied'])
dir_df

,directory,group,scan,run,mat_files,backups,empty_bups,last_working_bup,date_copied


In [21]:
def append_run_data(dataframe, run, scan=None):
    mat_files = len(list(run.glob("*.mat")))
    backups = list(run.glob('backup_*'))
    backup_type = {
        'empty': [],
        'partial': [],
        'full': [],
    }
    last_working = None

    
    for backup in backups:
        backup_contents = list(backup.glob('*'))
        backup_mat_files = list(backup.glob('*.mat'))
        if len(backup_contents) == 0:
            backup_type['empty'].append(backup)
        elif len(backup_mat_files) <= 2:
            backup_type['partial'].append(backup)
        else:
            backup_type['full'].append(backup)
            
        all_working = backup_type['partial'] + backup_type['full']
        all_working.sort()
        if len(all_working) > 0:
            last_working = all_working[-1].name.split('_')[-1]
        
    dataframe = dataframe.append({
        'directory': group.parent,
        'group': group.name,
        'scan': scan,
        'run': run.name,
        'mat_files': mat_files, 
        'backups': len(backups), 
        'empty_bups': len(backup_type['empty']), 
        'partial_bups': len(backup_type['partial']),
        'full_bups': len(backup_type['full']),
        'last_working_bup': last_working, 
        'date_copied': str(datetime.fromtimestamp(run.stat().st_ctime))
    }, ignore_index=True)
    return dataframe

In [30]:
dir_df = pd.DataFrame(columns=['directory', 'group', 'scan', 'run', 'mat_files', 'backups', 
                               'empty_bups', 'partial_bups', 'full_bups', 'last_working_bup', 'date_copied'])

for group in groups:
    # Skip if uninteresting
    if group.name in skippable_groups:
        continue
        
    print(f'\n{group.name}')
    
    for i, run in enumerate(group.glob('*')):
        if run.name in skippable_runs:
            continue
            
        backups = list(run.glob('backup_*'))
        scan_runs = list(run.glob('alpha_yz*'))

        # if there are backups it's probably a run
        if len(backups) > 0:
            dir_df = append_run_data(dir_df, run)
        
        # if there are folders names alpha_yz then it's a scan, and 
        # each scan-subfolder needs to be added separately
        elif len(scan_runs) > 0:
            for scan_run in scan_runs:
                dir_df = append_run_data(dir_df, scan_run, scan=run.name)
        


lowdens_anglescan

highdens_anglescan

lowdens_anglescan

magnum

middens_anglescan

middens_anglescan_failed

test

highdens_anglescan

lowdens_anglescan

middens_anglescan

rprobe_anomaly


In [31]:
dir_df

,directory,group,scan,run,mat_files,backups,empty_bups,partial_bups,full_bups,last_working_bup,date_copied
0,/home/jleland/data/external/spice/marconi,lowdens_anglescan,angled_tiltscan,alpha_yz_-9.0,98,2,0,0,2,20190809-0212,2019-08-09 02:12:28
1,/home/jleland/data/external/spice/marconi,lowdens_anglescan,angled_tiltscan,alpha_yz_-1.0,97,2,0,0,2,20190810-1039,2019-08-10 10:39:40
2,/home/jleland/data/external/spice/marconi,lowdens_anglescan,angled_tiltscan,alpha_yz_-2.0,98,2,0,0,2,20190810-1039,2019-08-10 10:39:33
3,/home/jleland/data/external/spice/marconi,lowdens_anglescan,angled_tiltscan,alpha_yz_-4.0,98,2,0,0,2,20190809-1402,2019-08-09 14:02:56
4,/home/jleland/data/external/spice/marconi,lowdens_anglescan,angled_tiltscan,alpha_yz_-10.0,98,2,0,0,2,20190809-0029,2019-08-09 00:29:44
...,...,...,...,...,...,...,...,...,...,...,...
414,/home/jleland/data/external_big/spice/marconi_...,middens_anglescan,sprobe_tiltscan,alpha_yz_-4.0,0,0,0,0,0,None,2020-01-22 16:57:17.562781
415,/home/jleland/data/external_big/spice/marconi_...,middens_anglescan,sprobe_tiltscan,alpha_yz_-45.0,0,0,0,0,0,None,2020-01-22 17:01:16.852456
416,/home/jleland/data/external_big/spice/marconi_...,middens_anglescan,sprobe_tiltscan,alpha_yz_-6.0,0,0,0,0,0,None,2020-01-22 16:23:53.284339
417,/home/jleland/data/external_big/spice/marconi_...,middens_anglescan,sprobe_tiltscan,alpha_yz_-8.0,0,0,0,0,0,None,2020-01-22 16:23:53.284770


In [35]:
dir_df.to_csv('spice_directory_info')